In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('string-network-1.csv')
df.head()

,node1,node2,node1_string_id,node2_string_id,homology,experimentally_determined_interaction,database_annotated,automated_textmining,combined_score
0,AP1B1,HIP1R,9606.ENSP00000350199,9606.ENSP00000253083,0,0.131,0.6,0,0.637
1,AP1B1,CLVS2,9606.ENSP00000350199,9606.ENSP00000275162,0,0.000,0.6,0,0.600
2,ASPSCR1,UFD1L,9606.ENSP00000306625,9606.ENSP00000263202,0,0.438,0.0,0,0.438
3,ASPSCR1,UBXN7,9606.ENSP00000306625,9606.ENSP00000296328,0,0.472,0.0,0,0.472
4,ASPSCR1,NSFL1C,9606.ENSP00000306625,9606.ENSP00000418529,0,0.747,0.0,0,0.747


In [7]:
# get list of nodes

nodes = df.node1.unique()

In [8]:
nodes

array(['AP1B1', 'ASPSCR1', 'ATP6V1A', 'ATP6V1B2', 'ATP6V1D', 'ATP6V1E1',
       'ATP6V1F', 'ATP6V1G1', 'ATP6V1H', 'B2M', 'CALR', 'CD1B', 'CD1D',
       'CD8A', 'CHMP2A', 'CHMP2B', 'CHMP3', 'CHMP4B', 'CLVS2', 'DERL1',
       'GGA1', 'HGS', 'HIP1R', 'HLA-A', 'HLA-B', 'HLA-C', 'HLA-E',
       'HLA-G', 'KLRC1', 'KLRD1', 'KPNA1', 'KPNA2', 'KPNB1', 'NSFL1C',
       'NUTF2', 'RAB5A', 'RAB7A', 'RAN', 'STAM', 'TSG101', 'UBXN7',
       'UFD1L', 'VCP', 'VPS33B', 'XPO1', 'XPO5'], dtype=object)

In [9]:
# build dict of node inputs

node_inputs = {}
for node in nodes:
    inputs = list(df[df.node1 == node].node2)
    node_inputs[node] = inputs

In [10]:
# construct boolean rules as list
rules = []
for key, value in node_inputs.items():
    rule = key + ' *= '
    rule = rule + ' and '.join(value)
    rules.append(rule)

In [11]:
# construct initial conditions
initial_conditions = []
for node in nodes:
    initial_condition = node + ' = Random'
    initial_conditions.append(initial_condition)

In [12]:
# export model
definition = '#initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#rules\n'+'\n'.join(rules)
fp = open('model.txt', 'w')
fp.write(definition)
fp.close()

In [13]:
inp = file('model.txt').read()

In [65]:
#  print(inp)

In [34]:
# write export helper functions

def string2definition(tabular_text_output, initial_node_value, out='model.txt'):
    df = pd.read_csv(tabular_text_output)
    
    # get list of nodes
    nodes = df.node1.unique()
    
    # build dict of node inputs
    node_inputs = {}
    for node in nodes:
        inputs = list(df[df.node1 == node].node2)
        node_inputs[node] = inputs
        
    # construct boolean rules as list
    rules = []
    for key, value in node_inputs.items():
        rule = key + ' *= '
        rule = rule + ' and '.join(value)
        rules.append(rule)
        
    # construct initial conditions
    initial_conditions = []
    for node in nodes:
        initial_condition = node + ' = ' + initial_node_value
        initial_conditions.append(initial_condition)
        
    # definition
    definition = '#initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#rules\n'+'\n'.join(rules)

#     # export
#     fp = open(out, 'w+')
#     fp.write(definition)
#     fp.close()
    
    return definition
        

In [45]:
def add_process_edgelist(definition, edgelist, initial_process_value):
    df = pd.read_csv(edgelist)
    
    # get list of nodes
    nodes = df.process.unique()

    # build dict of node inputs
    node_inputs = {}
    for node in nodes:
        inputs = list(df[df.process == node].node)
        node_inputs[node] = inputs
        
    # construct boolean rules as list
    rules = []
    for key, value in node_inputs.items():
        rule = key + ' *= '
        rule = rule + ' and '.join(value)
        rules.append(rule)
        
    # construct initial conditions
    initial_conditions = []
    for node in nodes:
        initial_condition = node + ' = ' + initial_process_value
        initial_conditions.append(initial_condition)
        
    # definition
    return definition + '\n\n#process node initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#process node rules\n'+'\n'.join(rules)


In [63]:
def add_mtb_edgelist(definition, edgelist, initial_mtb_value):
    df = pd.read_csv(edgelist)

    # get list of nodes
    target_nodes = df.node.unique()
    mtb_nodes = df.mtb.unique()
    
    # build dict of node inputs
    node_inputs = {}
    for node in target_nodes:
        inputs = list(df[df.node == node].mtb)
        node_inputs[node] = inputs
    
    # construct boolean rules as list
    rules = []
    for key, value in node_inputs.items():
        rule = key + ' *= ' + key
        rule = rule + ' and not (' +' or '.join(value) + ')'
        rules.append(rule)

    # construct initial conditions
    initial_conditions = []
    for node in mtb_nodes:
        initial_condition = node + ' = ' + initial_mtb_value
        initial_conditions.append(initial_condition)
        
    # definition
    return definition + '\n\n#mtb node initial conditions\n'+'\n'.join(initial_conditions)+'\n\n#mtb update rules\n'+'\n'.join(rules)

    

In [64]:
# print add_mtb_edgelist(definition, 'mtb-edgelist.csv', 'True')


#initial conditions
AP1B1 = Random
ASPSCR1 = Random
ATP6V1A = Random
ATP6V1B2 = Random
ATP6V1D = Random
ATP6V1E1 = Random
ATP6V1F = Random
ATP6V1G1 = Random
ATP6V1H = Random
B2M = Random
CALR = Random
CD1B = Random
CD1D = Random
CD8A = Random
CHMP2A = Random
CHMP2B = Random
CHMP3 = Random
CHMP4B = Random
CLVS2 = Random
DERL1 = Random
GGA1 = Random
HGS = Random
HIP1R = Random
HLA-A = Random
HLA-B = Random
HLA-C = Random
HLA-E = Random
HLA-G = Random
KLRC1 = Random
KLRD1 = Random
KPNA1 = Random
KPNA2 = Random
KPNB1 = Random
NSFL1C = Random
NUTF2 = Random
RAB5A = Random
RAB7A = Random
RAN = Random
STAM = Random
TSG101 = Random
UBXN7 = Random
UFD1L = Random
VCP = Random
VPS33B = Random
XPO1 = Random
XPO5 = Random

#rules
NUTF2 *= KPNA2 and KPNB1 and RAN
ATP6V1D *= ATP6V1G1 and ATP6V1B2 and ATP6V1E1 and ATP6V1H and ATP6V1F and ATP6V1A
CD1D *= CD1B and B2M
ATP6V1F *= ATP6V1D and ATP6V1E1 and ATP6V1A and ATP6V1B2 and UBXN7 and ATP6V1H and ATP6V1G1
ATP6V1A *= ATP6V1D and ATP6V1E1 and ATP6V1G1 